In [1]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from sklearn import preprocessing
from transformers import AutoTokenizer, TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [2]:
# 훈련 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/multinli.train.ko.tsv", filename="multinli.train.ko.tsv")
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/snli_1.0_train.ko.tsv", filename="snli_1.0_train.ko.tsv")

# 검증 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.dev.ko.tsv", filename="xnli.dev.ko.tsv")

# 테스트 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.test.ko.tsv", filename="xnli.test.ko.tsv")


('xnli.test.ko.tsv', <http.client.HTTPMessage at 0x28acbf76e40>)

In [3]:
train_snli = pd.read_csv("snli_1.0_train.ko.tsv", sep='\t', quoting=3)
train_xnli = pd.read_csv("multinli.train.ko.tsv", sep='\t', quoting=3)
val_data = pd.read_csv("xnli.dev.ko.tsv", sep='\t', quoting=3)
test_data = pd.read_csv("xnli.test.ko.tsv", sep='\t', quoting=3)


In [5]:
train_xnli[:5]

,sentence1,sentence2,gold_label
0,개념적으로 크림 스키밍은 제품과 지리라는 두 가지 기본 차원을 가지고 있다.,제품과 지리학은 크림 스키밍을 작동시키는 것이다.,neutral
1,시즌 중에 알고 있는 거 알아? 네 레벨에서 다음 레벨로 잃어버리는 거야 브레이브스...,사람들이 기억하면 다음 수준으로 물건을 잃는다.,entailment
2,우리 번호 중 하나가 당신의 지시를 세밀하게 수행할 것이다.,우리 팀의 일원이 당신의 명령을 엄청나게 정확하게 실행할 것이다.,entailment
3,어떻게 아세요? 이 모든 것이 다시 그들의 정보다.,이 정보는 그들의 것이다.,entailment
4,"그래, 만약 네가 테니스화 몇 개를 사러 간다면, 나는 왜 그들이 100달러대에서 ...",테니스화의 가격은 다양하다.,neutral


In [8]:
# 결합 후 섞기
# train_data = train_snli.append(train_xnli)
train_data = pd.concat([train_snli, train_xnli])
train_data = train_data.sample(frac=1)

In [9]:
train_data.head()

,sentence1,sentence2,gold_label
248002,8명의 소녀들이 노란색 유니폼을 입고 밖에서 치어리딩을 하고 있다.,이 소녀들은 각각 휠체어를 탄다.,contradiction
7375,파란색과 흰색 옷을 입은 축구선수가 팀 동료에게 공을 차주고 있고 다른 팀 동료들과...,공은 테니스 공이다.,contradiction
308490,"페너는 턱수염을 쪼개며 활짝 웃으며 소리쳤다. ""저 해안은 경주였어!","페너는 인상을 찌푸리며 ""그건 지루한 경주였다.",contradiction
6721,난로 위에서 음식 냄비를 휘젓는 키 큰 남자.,난로에서 요리하는 키 큰 남자.,entailment
156467,하지만 보통 9월 말까지 모든 것이 이루어진다는 것은 기본적으로 어떤 것도 재배하는...,9월 말 이후에는 아무것도 수확되지 않는다.,neutral


In [10]:
def drop_na_and_duplciates(df):
  df = df.dropna()
  df = df.drop_duplicates()
  df = df.reset_index(drop=True)
  return df


In [11]:
# 결측값 및 중복 샘플 제거
train_data = drop_na_and_duplciates(train_data)
val_data = drop_na_and_duplciates(val_data)
test_data = drop_na_and_duplciates(test_data)


In [12]:
print('훈련용 샘플 개수 :',len(train_data))
print('검증용 샘플 개수 :',len(val_data))
print('테스트용 샘플 개수 :',len(test_data))


훈련용 샘플 개수 : 941814
검증용 샘플 개수 : 2490
테스트용 샘플 개수 : 5010


## GPT 로드

In [13]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='<s>', eos_token='</s>', pad_token='<pad>')

In [14]:
# 정해진 정수값 확인
print(tokenizer.decode(0))
print(tokenizer.decode(1))
print(tokenizer.decode(2))
print(tokenizer.decode(3))
print(tokenizer.decode(4))


<s>
</s>
<usr>
<pad>
<sys>


In [15]:
max_seq_len = 128

sent1 = '모든 섬사람들이 알고 있듯이, 가장 멋진 만과 해변들 중 많은 것들은 보트로만 도달할 수 있다.'
sent2 = '보트는 일부 지역으로 가는 유일한 여행 수단이다.'

print('문장1 :',sent1)
print('문장2 :',sent2)


문장1 : 모든 섬사람들이 알고 있듯이, 가장 멋진 만과 해변들 중 많은 것들은 보트로만 도달할 수 있다.
문장2 : 보트는 일부 지역으로 가는 유일한 여행 수단이다.


위의 두 개의 문장으로부터 KoGPT-2에 넣을 입력으로 전처리를 진행해봅시다. KoGPT-2가 두 개의 서로 다른 문장임을 인식할 수 있도록 힌트를 줄 필요가 있습니다. 저자는 각 문장의 시작과 끝에 KoGPT-2의 시작 토큰과 종료 토큰을 붙이는 방식을 택했습니다. 그리고 입력이 완전히 끝났다는 것을 알려주기 위해서 <unused0>라는 사용 용도가 정해져 있지 않은 KoGPT-2의 스페셜 토큰을 사용하였습니다. 그 후 배치 연산을 위해 패딩을 해줍니다. KoGPT-2의 패딩 토큰은 정수 3입니다.

In [16]:
bos_token = [tokenizer.bos_token]
eos_token = [tokenizer.eos_token]

# 첫번째 문장의 앞과 뒤에 시작 토큰 <s>과 종료 토큰 </s>으로 감싼다.
sent1_tokens = bos_token + tokenizer.tokenize(sent1) + eos_token

# 두번째 문장의 앞과 뒤에 시작 토큰 <s>과 종료 토큰 </s>으로 감싼다. 그 후 <unused0>를 붙인다.
sent2_tokens = bos_token + tokenizer.tokenize(sent2) + eos_token + ['<unused0>']

# 두 개의 문장을 연달아 이어붙인 후 정수 인코딩을 수행한다.
tokens = sent1_tokens + sent2_tokens
input_id = tokenizer.convert_tokens_to_ids(tokens)
print('정수 인코딩 전:', tokens)
print('정수 인코딩 후:', input_id)

# 최대 길이로 패딩
input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]
print('패딩 후:', input_id)


정수 인코딩 전: ['<s>', '▁모든', '▁섬', '사람들이', '▁알고', '▁있듯이,', '▁가장', '▁멋진', '▁만과', '▁해변', '들', '▁중', '▁많은', '▁것들은', '▁보', '트로', '만', '▁도달할', '▁수', '▁있다.', '</s>', '<s>', '▁보', '트는', '▁일부', '▁지역으로', '▁가는', '▁유일한', '▁여행', '▁수단이', '다.', '</s>', '<unused0>']
정수 인코딩 후: [0, 9548, 9709, 34539, 12487, 42370, 9278, 43719, 34766, 23545, 7285, 9044, 9366, 24860, 9049, 11714, 7489, 48699, 9025, 10960, 1, 0, 9049, 11943, 9616, 14303, 11318, 13382, 12079, 26626, 9016, 1, 9]
패딩 후: [    0  9548  9709 34539 12487 42370  9278 43719 34766 23545  7285  9044
  9366 24860  9049 11714  7489 48699  9025 10960     1     0  9049 11943
  9616 14303 11318 13382 12079 26626  9016     1     9     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3 

전처리 결과는 위와 같습니다. 각 문장의 시작에는 시작 토큰인 <s>가 붙어있으며 정수로는 0입니다. 각 문장의 끝에는 종료 토큰인 </s>가 붙어있으며 정수로는 1입니다. 이에 따라 두 개의 문장의 앞, 뒤에는 0과 1이 있습니다. 그리고 입력이 완전히 끝나면 <unused0>에 해당하는 정수인 9가 부착됩니다. 그리고 최대 길이를 128로 정하였으므로 128의 길이로 일치시켜주기 위해서 패딩 토큰인 정수 3이 채워집니다.

위 과정을 convert_examples_to_features 라는 함수로 만들고, 훈련 데이터의 첫번째 샘플을 가지고 임의로 진행했던 전처리를 훈련 데이터, 검증 데이터, 테스트 데이터에 대해서 모두 진행해봅시다.

In [17]:
def convert_examples_to_features(sent_list1, sent_list2, max_seq_len, tokenizer):

    input_ids = []

    for sent1, sent2 in tqdm(zip(sent_list1, sent_list2), total=len(sent_list1)):
        bos_token = [tokenizer.bos_token]
        eos_token = [tokenizer.eos_token]
        sent1_tokens = bos_token + tokenizer.tokenize(sent1) + eos_token
        sent2_tokens = bos_token + tokenizer.tokenize(sent2) + eos_token + ['<unused0>']
        tokens = sent1_tokens + sent2_tokens
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        input_ids.append(input_id)

    input_ids = np.array(input_ids, dtype=int)

    return input_ids


In [18]:
# 훈련데이터에 대한 전처리
X_train = convert_examples_to_features(train_data['sentence1'], train_data['sentence2'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 941814/941814 [02:01<00:00, 7776.19it/s]


훈련 데이터의 첫번째 샘플에 대한 정수 인코딩, 정수 인코딩을 기존의 문자열로 복원한 결과는 다음과 같습니다. (데이터가 섞일 때는 랜덤이므로 저자와 첫번째 샘플은 다를 수 있습니다.)

In [19]:
# 최대 길이: 128
input_id = X_train[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))


단어에 대한 정수 인코딩 : [    0  9253  9902 21725  9136 25625 21075  8662  8137 14087 23874  9407
 20350  7301  8137  9676 10960     1     0  9018 21725  9177  9880 27198
  8368 10546  9731  9016     1     9     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3]
각 인코딩의 길이 : 128
정수 인코딩 복원 : <s> 8명의 소녀들이 노란색 유니폼을 입고 밖에서 치어리딩을 하고 있다.</s><s> 이 소녀들은 각각 휠체어를 탄다.</s><unused0><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

검증 데이터에 대해서도 전처리를 진행해봅시다.

In [20]:
X_val = convert_examples_to_features(val_data['sentence1'], val_data['sentence2'], max_seq_len=max_seq_len, tokenizer=tokenizer)


100%|██████████| 2490/2490 [00:00<00:00, 7190.13it/s]


In [21]:
# 최대 길이: 128
input_id = X_val[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))


단어에 대한 정수 인코딩 : [    0  9394  9871  9135  8718 14364 10063  8013 37144  9265 12583  8006
 25856   377     1     0  9258 10192  9848 11001 10644 10396 18796 20485
 37472  9134 35673  9539 18174     1     9     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3]
각 인코딩의 길이 : 128
정수 인코딩 복원 : <s> 그리고 그가 말했다, "엄마, 저 왔어요."</s><s> 그는 학교 버스가 그를 내려주자마자 엄마에게 전화를 걸었다.</s><unused0><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

테스트 데이터에 대해서 전처리를 진행해봅시다.

In [22]:
X_test = convert_examples_to_features(test_data['sentence1'], test_data['sentence2'], max_seq_len=max_seq_len, tokenizer=tokenizer)


100%|██████████| 5010/5010 [00:00<00:00, 7357.80it/s]


contradiction, entailment, neutral과 같이 문자열로 구성된 레이블에 대해서도 정수 인코딩을 진행합니다.

In [25]:
train_label = train_data['gold_label'].tolist()
val_label = val_data['gold_label'].tolist()
test_label = test_data['gold_label'].tolist()

idx_encode = preprocessing.LabelEncoder()
# print(type(idx_encode))
idx_encode.fit(train_label) # 검증, 테스트는 하지 않아도 됨. 여기에 다 포함되기 때문에

# 고유한 정수로 변환. 고유한 정수라 해봤자 3개니까 0,1,2 이다.
y_train = idx_encode.transform(train_label)
y_val = idx_encode.transform(val_label) 
y_test = idx_encode.transform(test_label)


label_idx = dict(zip(list(idx_encode.classes_), idx_encode.transform(list(idx_encode.classes_))))
idx_label = {value: key for key, value in label_idx.items()}
print('각 레이블과 정수 :', label_idx)


각 레이블과 정수 : {'contradiction': 0, 'entailment': 1, 'neutral': 2}


In [26]:
print('변환 전 :', test_label[:5])
print('변환 후 :',y_test[:5])

변환 전 : ['contradiction', 'entailment', 'neutral', 'neutral', 'entailment']
변환 후 : [0 1 2 2 1]


## GPT 출력 이해하기

koGPT-2를 이용해 모델을 구현하기 위해서는 koGPT-2의 출력을 이해할 필요가 있습니다. 우선, 한국어 GPT-2인 skt/kogpt2-base-v2를 로드해봅시다.

In [27]:
model = TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.1.attn.masked_bias', 'lm_head.weight', 'transformer.h.5.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.0.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from 

In [29]:
import tf_keras as keras

max_seq_len = 128

# input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
input_ids_layer = keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
outputs = model([input_ids_layer])


outputs에는 두 개의 출력이 존재하는데 인덱스 0을 확인해봅시다.

In [30]:
# 문장 길이만큼의 출력
print(outputs[0])

KerasTensor(type_spec=TensorSpec(shape=(None, 128, 768), dtype=tf.float32, name=None), name='tfgpt2_model/transformer/Reshape_2:0', description="created by layer 'tfgpt2_model'")


outputs[0]은 (배치 크기, 128, 768)의 크기를 가지는 텐서입니다. 이는 768차원의 벡터가 128개가 있다는 의미로 문장 길이 개수만큼의 출력을 얻었음을 의미합니다. 텍스트 분류 문제를 풀 경우에는 koGPT-2의 마지막 예측에 해당하는 벡터를 사용해야 합니다.

In [31]:
# 마지막 출력 벡터
print(outputs[0][:, -1])

KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='tf.__operators__.getitem/strided_slice:0', description="created by layer 'tf.__operators__.getitem'")


## 여기서 부터가 진짜. 위에는 OUTPUT에 어떤 정보가 있는지 확인 차원

서브클래싱 구현 방식으로 구현한 텍스트 분류 모델은 다음과 같습니다. GPT의 출력 중 outputs[0][:, -1]. 즉, 마지막 출력 벡터를 소프트맥스 함수가 활성화 함수로 설정된 출력층으로 연결합니다.

In [32]:
class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='softmax',
                                                name='classifier')

    def call(self, inputs):
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs[0][:, -1]
        prediction = self.classifier(cls_token)

        return prediction


In [33]:
model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2", num_labels=3)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.1.attn.masked_bias', 'lm_head.weight', 'transformer.h.5.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.0.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [ ]:
early_stopping = EarlyStopping(
    monitor="val_accuracy", 
    min_delta=0.001,
    patience=2)

model.fit(
    X_train, y_train, epochs=2, batch_size=32, validation_data = (X_val, y_val),
    callbacks = [early_stopping]
)

In [ ]:
results = model.evaluate(X_test, y_test, batch_size=1024)
print("test loss, test acc: ", results)